In [ ]:
# Install Feast
#==5.3.1 
#==0.10.5
!pip install feast==0.22.0 PyYAML-q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PyYAML-q (from versions: none)
ERROR: No matching distribution found for PyYAML-q


In [ ]:
!pip freeze | grep pyYAML

Create a feature repository at the root of our project. [Feast](https://feast.dev/) will create a configuration file for us and we're going to add an additional [features.py](https://github.com/GokuMohandas/mlops-course/blob/main/features/features.py) file to define our features.

> Traditionally, the feature repository would be it's own isolated repository that other services will use to read/write features from.

In [ ]:
%%bash
mkdir -p stores/feature
mkdir -p data
feast init --minimal --template local features
cd features
touch features.py

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage
The directory features contains an existing feature store repository that may cause a conflict



/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


```bash
features/
├── feature_store.yaml  - configuration
└── features.py         - feature definitions
```

In [ ]:
%%bash
FEATURE_STORE_YAML=features/feature_store.yaml
if test -f $FEATURE_STORE_YAML; then
    rm $FEATURE_STORE_YAML
fi
touch $FEATURE_STORE_YAML
echo "project: features" >> $FEATURE_STORE_YAML
echo "registry: ../stores/feature/registry.db" >> $FEATURE_STORE_YAML
echo "provider: local" >> $FEATURE_STORE_YAML
echo "online_store:" >> $FEATURE_STORE_YAML
echo "    path: ../stores/feature/online_store.db" >> $FEATURE_STORE_YAML
cat $FEATURE_STORE_YAML

project: features
registry: ../stores/feature/registry.db
provider: local
online_store:
    path: ../stores/feature/online_store.db


# Data ingestion

The first step is to establish connections with our data sources (databases, data warehouse, etc.). Feast requires it's data sources to either come from a file ([Parquet](https://databricks.com/glossary/what-is-parquet)), data warehouse ([BigQuery](https://cloud.google.com/bigquery)) or [Kinesis](https://aws.amazon.com/kinesis/)). We'll convert our generated features file from the DataOps pipeline (`features.json`) into a Parquet file, which is a column-major data format that allows fast feature retrieval and caching benefits (contrary to row-major data formats such as CSV where we have to traverse every single row to collect feature values).

In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from urllib.request import urlopen

In [ ]:
# Load projects 
url = "https://raw.githubusercontent.com/Samagra-Development/machine-learning-platform/main/datasets/projects.json"
projects = json.loads(urlopen(url).read())
df = pd.DataFrame(projects)
df["text"] = df.title + " " + df.description
df.drop(["title", "description"], axis=1, inplace=True)
df.head(5)

,id,created_on,tag,text
0,6,2020-02-20 06:43:18,computer-vision,Comparison between YOLO and RCNN on real world...
1,7,2020-02-20 06:47:21,computer-vision,"Show, Infer & Tell: Contextual Inference for C..."
2,9,2020-02-24 16:24:45,graph-learning,Awesome Graph Classification A collection of i...
3,15,2020-02-28 23:55:26,reinforcement-learning,Awesome Monte Carlo Tree Search A curated list...
4,19,2020-03-03 13:54:31,graph-learning,Diffusion to Vector Reference implementation o...


In [ ]:
# Format timestamp
df.created_on = pd.to_datetime(df.created_on)

In [ ]:
# Convert to parquet
DATA_DIR = Path(os.getcwd(), "data")
df.to_parquet(
    Path(DATA_DIR, "features.parquet"),
    compression=None,
    allow_truncated_timestamps=True,
)

# Feature definitions

Now that we have our data source prepared, we can define our features for the feature store.

In [ ]:
from datetime import datetime
from pathlib import Path
from feast import Entity, Feature, FeatureView, ValueType
from feast.data_source import FileSource
from google.protobuf.duration_pb2 import Duration

The first step is to define the location of the features (FileSource in our case) and the timestamp column for each data point.

In [ ]:
# Read data
START_TIME = "2020-02-17"
project_details = FileSource(
    path=str(Path(DATA_DIR, "features.parquet")),
    event_timestamp_column="created_on",
)

Next, we need to define the main entity that each data point pertains to. In our case, each project has a unique ID with features such as text and tags.

In [ ]:
# Define an entity
project = Entity(
    name="id",
    value_type=ValueType.INT64,
    description="project id",
)

Finally, we're ready to create a [FeatureView](https://docs.feast.dev/concepts/feature-views) that loads specific features (`features`), of various [value types](https://api.docs.feast.dev/python/feast.html?highlight=valuetype#feast.value_type.ValueType), from a data source (`input`) for a specific period of time (`ttl`).

In [ ]:
# Define a Feature View for each project
project_details_view = FeatureView(
    name="project_details",
    entities=["id"],
    ttl=Duration(
        seconds=(datetime.today() - datetime.strptime(START_TIME, "%Y-%m-%d")).days * 24 * 60 * 60
    ),
    features=[
        Feature(name="text", dtype=ValueType.STRING),
        Feature(name="tag", dtype=ValueType.STRING),
    ],
    online=True,
    input=project_details,
    tags={},
)

We need to place all of this code into our `features.py` file we created earlier.

In [ ]:
%%bash
FEATURES_PY=features/features.py
if test -f $FEATURES_PY; then
    rm $FEATURES_PY
fi
touch $FEATURES_PY
echo -e 'from datetime import datetime
from pathlib import Path

from feast import Entity, Feature, FeatureView, ValueType
from feast.data_source import FileSource
from google.protobuf.duration_pb2 import Duration


# Read data
START_TIME = "2020-02-17"
project_details = FileSource(
    path="/content/data/features.parquet",
    event_timestamp_column="created_on",
)

# Define an entity for the project
project = Entity(
    name="id",
    value_type=ValueType.INT64,
    description="project id",
)

# Define a Feature View for each project
# Can be used for fetching historical data and online serving
project_details_view = FeatureView(
    name="project_details",
    entities=["id"],
    ttl=Duration(
        seconds=(datetime.today() - datetime.strptime(START_TIME, "%Y-%m-%d")).days * 24 * 60 * 60
    ),
    features=[
        Feature(name="text", dtype=ValueType.STRING),
        Feature(name="tag", dtype=ValueType.STRING),
    ],
    online=True,
    input=project_details,
    tags={},
)' >> $FEATURES_PY

Once we've defined our feature views, we can `apply` it to push a version controlled definition of our features to the registry for fast access. It will also configure our registry and online stores that we've defined in our `feature_store.yaml`.

In [ ]:
%%bash
cd features
feast apply

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
Traceback (most recent call last):
  File "/usr/local/bin/feast", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.7/dist-packages/

# Historical features

Once we've registered our feature definition, along with the data source, entity definition, etc., we can use it to fetch historical features. 

In [ ]:
import pandas as pd
from feast import FeatureStore

In [ ]:
# Identify entities
project_ids = df.id[0:3].to_list()
now = datetime.now()
timestamps = [datetime(now.year, now.month, now.day)]*len(project_ids)
entity_df = pd.DataFrame.from_dict({"id": project_ids, "event_timestamp": timestamps})
entity_df.head()

,id,event_timestamp
0,6,2022-07-08
1,7,2022-07-08
2,9,2022-07-08


In [ ]:
# Get historical features
store = FeatureStore(repo_path="features")
training_df = store.get_historical_features(
    entity_df=entity_df,
    feature_refs=["project_details:text", "project_details:tag"],
).to_df()
training_df.head()

,event_timestamp,id,project_details__text,project_details__tag
0,2022-07-08 00:00:00+00:00,6,Comparison between YOLO and RCNN on real world...,computer-vision
1,2022-07-08 00:00:00+00:00,7,"Show, Infer & Tell: Contextual Inference for C...",computer-vision
2,2022-07-08 00:00:00+00:00,9,Awesome Graph Classification A collection of i...,graph-learning


# Materialize

For online inference, we want to retrieve features very quickly via our online store, as opposed to fetching them from slow joins. However, the features are not in our online store just yet, so we'll need to materialize them first.

In [ ]:
%%bash
cd features
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME

Materializing 1 feature views to 2022-07-08 12:16:14+00:00 into the sqlite online store.

project_details from 2022-07-08 12:05:16+00:00 to 2022-07-08 12:16:14+00:00:


/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
0it [00:00, ?it/s]


# Online features

In [ ]:
# Get online features
store = FeatureStore(repo_path="features")
feature_vector = store.get_online_features(
    feature_refs=["project_details:text", "project_details:tag"],
    entity_rows=[{"id": 6}],
).to_dict()
feature_vector

{'id': [6],
 'project_details__tag': ['computer-vision'],
 'project_details__text': ['Comparison between YOLO and RCNN on real world videos Bringing theory to experiment is cool. We can easily train models in colab and find the results in minutes.']}

In [ ]:
#from google.colab.output import eval_js
#host = eval_js("google.colab.kernel.proxyPort(8888)")

#!nohup feast ui &
#print(f"Open the Web UI at {host}")

nohup: appending output to 'nohup.out'
Open the Web UI at https://8xxe88rhhk-496ff2e9c6d22116-8888-colab.googleusercontent.com/
